<a href="https://colab.research.google.com/github/kangdoung/Traffic_detection/blob/main/VietAI_Final_Tracking_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Nov 14 09:10:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Mount from drive to colab disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Clone github chứa code Deepsort Và một số xử lý.

#Refereces: Github này của một người bạn của em code trong lúc làm HCMC AI City. @Nguyễn Hoàng Quân FIT HCMUS

In [ ]:
!git clone https://github.com/wan2000/DeepSORT.git

Cloning into 'DeepSORT'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 48 (delta 16), reused 35 (delta 9), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [ ]:
%cd DeepSORT

/content/DeepSORT


In [ ]:
import cv2
import numpy as np
import json
import random

In [ ]:
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


#Định nghĩa threshold cho từng loại class: Theo nhóm nhận thấy thì object nhỏ tỉ lệ cao sẽ có confidence scores nhỏ.

In [ ]:
# Options
threshold = {
    1 : 0.0,
    2 : 0.2,
    3 : 0.3,
    4 : 0.4
}

#Tải weight của model DeepSort

In [ ]:
!mkdir model_data
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1bB66hP9voDXuoBoaCcKYY7a8IYzMMs4P', dest_path='model_data/mars-small128.pb', unzip=False)

#Một số hàm để đọc dữ liệu
#References: baseline của BTC - https://github.com/hcmcaic/ai-challenge-2020

In [ ]:
import json
def load_zone_anno(json_filename):
  """
  Load the json with ROI and MOI annotation.

  """
  with open(json_filename) as jsonfile:
    dd = json.load(jsonfile)
    polygon = [(int(x), int(y)) for x, y in dd['shapes'][0]['points']]
    paths = {}
    for it in dd['shapes'][1:]:
      kk = str(int(it['label'][-2:]))
      paths[kk] = [(int(x), int(y)) for x, y
              in it['points']]
  return polygon, paths

In [ ]:
import bb_polygon
def check_bbox_intersect_polygon(polygon, bbox):
  """
  
  Args:
    polygon: List of points (x,y)
    bbox: A tuple (xmin, ymin, xmax, ymax)
  
  Returns:
    True if the bbox intersect the polygon
  """
  x1, y1, x2, y2 = bbox
  bb = [(x1,y1), (x2, y1), (x2,y2), (x1,y2)]
  return bb_polygon.is_bounding_box_intersect(bb, polygon)

#Định nghĩa hàm NMS (Non Maximum Suppression) để loại bỏ box đè nhau trong cùng 1 object.

In [ ]:
def hard_nms(boxes, label, scores, iou_thresh=None):
    """The basic hard non-maximum suppression.
    Args:
        dets: detection with shape (num, 5) and format [x1, y1, x2, y2, score].
        iou_thresh: IOU threshold,
    Returns:
        numpy.array: Retained boxes.
    """
    iou_thresh = iou_thresh or 0.5
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        intersection = w * h
        overlap = intersection / (areas[i] + areas[order[1:]] - intersection)

        inds = np.where(overlap <= iou_thresh)[0]
        order = order[inds + 1]

    return boxes[keep], label[keep], scores[keep]
def hard_nms_per_class(boxes, labels, scores, iou_thresh = 0.3):
  new_boxes = []
  new_labels = []
  new_scores = []
  for l in range(1, 5):
    indices = np.where(labels == l)[0]
    temp_boxes = boxes[indices]
    temp_labels = labels[indices]
    temp_scores = scores[indices]
    temp_boxes, temp_labels, temp_scores = hard_nms(temp_boxes, temp_labels, temp_scores, iou_thresh)
    if l == 1:
      new_boxes, new_labels, new_scores = temp_boxes, temp_labels, temp_scores
    else:
      new_boxes = np.concatenate([new_boxes, temp_boxes])
      new_labels = np.concatenate([new_labels, temp_labels])
      new_scores = np.concatenate([new_scores, temp_scores])
  return new_boxes, new_labels, new_scores


#Hàm trả về label của một tracking_box, ở hàm này tụi mình sử dụng IoU metrics.

In [ ]:
def find_tracking_label(tracking_box, boxes, iou_thresh = 0.3):
    '''
    Args:
        tracking_box: shape (4) and format [x_min, y_min, x_max, y_max]
        boxes: detection with shape (num, 5) and format [x1, y1, x2, y2, label]
    Returns:
        label of the tracking box
    '''
    if len(boxes) == 0:
        return -1
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    x_min, y_min, x_max, y_max = tracking_box
    tracking_area = (x_max - x_min + 1) * (y_max - y_min + 1)

    xx1 = np.maximum(x_min, x1)
    yy1 = np.maximum(y_min, y1)
    xx2 = np.minimum(x_max, x2)
    yy2 = np.minimum(y_max, y2)

    w = np.maximum(0.0, xx2 - xx1 + 1)
    h = np.maximum(0.0, yy2 - yy1 + 1)
    intersection = w * h

    iou = intersection / (areas + tracking_area - intersection)
    
    if np.max(iou) < iou_thresh:
        return -1
    return boxes[np.argmax(iou), -1]

#Hàm tính độ tương đương cosin giữa 2 vectors.

In [ ]:
def cosin_similarity(a2d, b2d):  
  a = np.array((a2d[1][0] - a2d[0][0], a2d[1][1] - a2d[0][1]))
  b = np.array((b2d[1][0] - b2d[0][0], b2d[1][1] - b2d[0][1]))
  return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

#Sử lý từng camera và in ra files tracking_{cam_num}.txt theo thứ tự sau:
#1. Đọc dữ liệu detections từ file {cam_num}.txt, file json và file zip chứa các frame ảnh.
#2. Khởi tạo model DeepSort với weight pretrained đã được download ở trên
#3. Mỗi frame, đưa mỗi box vào tracking để update tracking rồi in ra file txt theo định dạng: {track_id, x_min, y_min, x_max, y_max, frame_id, label} có nghĩa: id của tracking, {x_min, y_min, x_max, y_max} là box của tracking, id của frame và label của box tracking đấy.
#4. Sau đó, chạy file moi_counting.py ở github được đề cập ở trên (https://github.com/wan2000/DeepSORT/tree/master/deep_sort) để lấy files result_{cam_num}.txt là kết quả cho camera thứ cam_num:


In [ ]:
import os
from tqdm.notebook import tqdm

for idx in range(1, 11):
    cam_num = 'cam_%02d' % idx
    print("Copying and unzipping {}".format(cam_num))
    !rm -rf datasets
    !mkdir datasets
    !mkdir datasets/{cam_num}
    !cp '/content/drive/My Drive/VietAI - Final Project/detections/new_detection/retrain_2/{cam_num}.txt' datasets/{cam_num}.txt
    !cp '/content/drive/My Drive/VietAI - Final Project/json_file/{cam_num}.json' datasets/{cam_num}.json
    !cp "/content/drive/My Drive/VietAI - Final Project/New ZIP/{cam_num}.zip" 'datasets/{cam_num}.zip'
    %cd datasets
    !unzip -q {cam_num}.zip -d {cam_num}
    !rm {cam_num}.zip
    %cd ..

    f = open('datasets/{}.txt'.format(cam_num))
    events = f.readlines()

    polygon, paths = load_zone_anno('datasets/{}.json'.format(cam_num))

    # Definition of the parameters
    max_cosine_distance = 0.5
    nn_budget = None
    nms_max_overlap = 1.0

    #initialize deep sort
    model_filename = 'model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)

    tracker = Tracker(metric, max_age=10)

    track_label_counter = {}
    track_label = {}

    track_dict = {}

    event_id = 0

    f_tracking = open('/content/drive/My Drive/VietAI - Final Project/tracking_baseline/new_tracking/tracking_{}.txt'.format(cam_num), 'w')

    image_files = os.listdir('datasets/{}'.format(cam_num))
    image_files.sort()
    cnt = 0
    for frame_id in tqdm(range(len(image_files)), disable=True):
      cnt += 1
      print(image_files[frame_id])
      if (cnt >= 5): break
    print('Processing {}'.format(cam_num))

    for frame_id in tqdm(range(len(image_files)), disable=True):
        # ret, frame = cap.read()
        # image = frame.copy()

        image = cv2.imread(os.path.join('datasets/{}'.format(cam_num), image_files[frame_id]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        image = np.array(image, np.float32) / 255.

        bboxes = []
        confidents = []
        labels = []
        dets = []

        while True:
            e = list(map(float, events[event_id].split(' ')))
            if e[0] != frame_id:
                break

            x_min, y_min, x_max, y_max = np.array(e[1:5], np.int32)
            score = e[5]
            label = int(e[6])
            if label != -1 and score > threshold[label]:
                bboxes.append((x_min, y_min, x_max, y_max))
                confidents.append(score)
                labels.append(label)
                dets.append([x_min, y_min, x_max, y_max, label])
            event_id += 1
            if event_id >= len(events):
                break

        bboxes = np.array(bboxes)
        confidents = np.array(confidents)
        labels = np.array(labels)
        dets = np.array(dets)

        if (frame_id <= 2):
          print('len before nms = ', len(bboxes))
        
        if len(bboxes) > 0:
            bboxes, labels, confidents = hard_nms(bboxes, labels, confidents, 0.3)
            # from tlbr to tlwh
            bboxes[:, 2] -= bboxes[:, 0]
            bboxes[:, 3] -= bboxes[:, 1]
        if (frame_id <= 2):
          print('len after nms = ', len(bboxes))

        features = encoder(image, bboxes)
        detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(bboxes, confidents, features)]

        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            x_min, y_min, x_max, y_max = track.to_tlbr()
            track_id = track.track_id

            label = find_tracking_label([x_min, y_min, x_max, y_max], dets)

            if track_id not in track_dict.keys():
                track_dict[track_id] = [(x_min, y_min, x_max, y_max, frame_id, label)]
                track_label_counter[track_id] = {
                    -1 : 0, 1 : 0, 2 : 0, 3 : 0, 4 : 0,
                }
            else:
                track_dict[track_id].append((x_min, y_min, x_max, y_max, frame_id, label))
            
            f_tracking.write('{} {} {} {} {} {} {}\n'.format(track_id, x_min, y_min, x_max, y_max, frame_id, label))

            track_label_counter[track_id][label] += 1
            for i in range(1,5):
                if track_label_counter[track_id][label] < track_label_counter[track_id][i]:
                    label = i
            track_label[track_id] = label

Copying and unzipping cam_01
/content/DeepSORT/datasets
/content/DeepSORT
0000.jpg
0001.jpg
0002.jpg
0003.jpg
0004.jpg
Processing cam_01
len before nms =  27
len after nms =  16


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f60ae51baf0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


len before nms =  34
len after nms =  20
len before nms =  39
len after nms =  21


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Copying and unzipping cam_02
/content/DeepSORT/datasets
/content/DeepSORT
0000.jpg
0001.jpg
0002.jpg
0003.jpg
0004.jpg
Processing cam_02
len before nms =  1
len after nms =  1


...

len before nms =  9
len after nms =  6
len before nms =  4
len after nms =  4
Copying and unzipping cam_09
/content/DeepSORT/datasets
/content/DeepSORT
0000.jpg
0001.jpg
0002.jpg
0003.jpg
0004.jpg
Processing cam_09
len before nms =  12
len after nms =  7


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f60aec1d518>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


len before nms =  14
len after nms =  6
len before nms =  8
len after nms =  6
Copying and unzipping cam_10
/content/DeepSORT/datasets
/content/DeepSORT
0000.jpg
0001.jpg
0002.jpg
0003.jpg
0004.jpg
Processing cam_10
len before nms =  3
len after nms =  3


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f60a4e60fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


len before nms =  1
len after nms =  1
len before nms =  1
len after nms =  1
